In [1]:
# from sklearn.metrics import average_precision_score
# def sklearn_mean_ap(preds, targs):
#     """
#     Difference from COCO is precision is not interpolated
#     targs: (n), preds: (nx4)
#     """
#     return np.mean([average_precision_score(targs==i,preds[:,i]) for i in range(4)])*2/3

In [2]:
import ast
import json
import os
import shutil

import albumentations
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom as dicom
import tensorflow as tf
import tensorflow_addons as tfa
import torch
from PIL import Image
from sklearn.metrics import average_precision_score
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras
from tqdm import tqdm

from augment import Aug, Aug_Crop, Aug_No_transform, Flip_Aug
from generator import Generator, GetModel

MOD = "EFFB4"

In [17]:
shutil.rmtree('/app/_data/npy_224')

In [3]:
df = pd.read_csv("/app/_data/predicted_crop_v1.csv")
list_wrong = df[(df["class"] != "negative") & (df["label"] == "none 1 0 0 1 1")][
    "id_image"
].tolist()
df = df.query("id_image not in @list_wrong").reset_index(drop=True)

In [4]:
with open("/app/_data/base_config.json", "r") as f:
    base_config = json.load(f)
base_config[MOD]["BATCH_SIZE"] = 30

In [5]:
labels = pd.read_csv("/app/_data/additional_dataset/labels_full.csv")
labels

,patientId,x,y,width,height,Target,modality,PatientSex,BodyPartExamined,PhotometricInterpretation,...,y_center,w,h,class,image,path,xmin,ymin,xmax,ymax
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0,CR,F,CHEST,MONOCHROME2,...,NaN,NaN,NaN,0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,/app/_data/additional_dataset/stage_2_train_im...,0.231538,0.152784,0.868654,0.751009
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0,CR,F,CHEST,MONOCHROME2,...,NaN,NaN,NaN,0,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,/app/_data/additional_dataset/stage_2_train_im...,0.063302,0.235900,0.883660,0.761328
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0,CR,M,CHEST,MONOCHROME2,...,NaN,NaN,NaN,0,00322d4d-1c29-4943-afc9-b6754be640eb,/app/_data/additional_dataset/stage_2_train_im...,0.114486,0.041784,0.890716,0.724560
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0,CR,M,CHEST,MONOCHROME2,...,NaN,NaN,NaN,0,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,/app/_data/additional_dataset/stage_2_train_im...,0.125391,0.111954,0.875462,0.863721
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1,CR,F,CHEST,MONOCHROME2,...,0.333496,0.208008,0.370117,0,00436515-870c-4b36-a041-de91049b9ab4,/app/_data/additional_dataset/stage_2_train_im...,0.185933,0.134582,0.805438,0.672207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,185.0,298.0,228.0,379.0,1,CR,M,CHEST,MONOCHROME2,...,0.476074,0.222656,0.370117,0,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,/app/_data/additional_dataset/stage_2_train_im...,0.136216,0.093138,0.869451,0.787371
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,NaN,NaN,NaN,NaN,0,CR,F,CHEST,MONOCHROME2,...,NaN,NaN,NaN,0,c1edf42b-5958-47ff-a1e7-4f23d99583ba,/app/_data/additional_dataset/stage_2_train_im...,0.172609,0.197233,0.895536,0.821171
30224,c1f6b555-2eb1-4231-98f6-50a963976431,NaN,NaN,NaN,NaN,0,CR,M,CHEST,MONOCHROME2,...,NaN,NaN,NaN,0,c1f6b555-2eb1-4231-98f6-50a963976431,/app/_data/additional_dataset/stage_2_train_im...,0.082085,0.100155,0.878707,0.936668
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,570.0,393.0,261.0,345.0,1,CR,F,CHEST,MONOCHROME2,...,0.552246,0.254883,0.336914,0,c1f7889a-9ea9-4acb-b64c-b737c929599a,/app/_data/additional_dataset/stage_2_train_im...,0.183922,0.191654,0.875833,0.825657


In [6]:
labels["class"] = labels["Target"].replace({0: "negative", 1: "positive"})
labels["Negative for Pneumonia"] = labels["Target"].replace({0: 1, 1: 0})
labels

,patientId,x,y,width,height,Target,modality,PatientSex,BodyPartExamined,PhotometricInterpretation,...,w,h,class,image,path,xmin,ymin,xmax,ymax,Negative for Pneumonia
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0,CR,F,CHEST,MONOCHROME2,...,NaN,NaN,negative,0004cfab-14fd-4e49-80ba-63a80b6bddd6,/app/_data/additional_dataset/stage_2_train_im...,0.231538,0.152784,0.868654,0.751009,1
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0,CR,F,CHEST,MONOCHROME2,...,NaN,NaN,negative,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,/app/_data/additional_dataset/stage_2_train_im...,0.063302,0.235900,0.883660,0.761328,1
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0,CR,M,CHEST,MONOCHROME2,...,NaN,NaN,negative,00322d4d-1c29-4943-afc9-b6754be640eb,/app/_data/additional_dataset/stage_2_train_im...,0.114486,0.041784,0.890716,0.724560,1
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0,CR,M,CHEST,MONOCHROME2,...,NaN,NaN,negative,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,/app/_data/additional_dataset/stage_2_train_im...,0.125391,0.111954,0.875462,0.863721,1
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1,CR,F,CHEST,MONOCHROME2,...,0.208008,0.370117,positive,00436515-870c-4b36-a041-de91049b9ab4,/app/_data/additional_dataset/stage_2_train_im...,0.185933,0.134582,0.805438,0.672207,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,185.0,298.0,228.0,379.0,1,CR,M,CHEST,MONOCHROME2,...,0.222656,0.370117,positive,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,/app/_data/additional_dataset/stage_2_train_im...,0.136216,0.093138,0.869451,0.787371,0
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,NaN,NaN,NaN,NaN,0,CR,F,CHEST,MONOCHROME2,...,NaN,NaN,negative,c1edf42b-5958-47ff-a1e7-4f23d99583ba,/app/_data/additional_dataset/stage_2_train_im...,0.172609,0.197233,0.895536,0.821171,1
30224,c1f6b555-2eb1-4231-98f6-50a963976431,NaN,NaN,NaN,NaN,0,CR,M,CHEST,MONOCHROME2,...,NaN,NaN,negative,c1f6b555-2eb1-4231-98f6-50a963976431,/app/_data/additional_dataset/stage_2_train_im...,0.082085,0.100155,0.878707,0.936668,1
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,570.0,393.0,261.0,345.0,1,CR,F,CHEST,MONOCHROME2,...,0.254883,0.336914,positive,c1f7889a-9ea9-4acb-b64c-b737c929599a,/app/_data/additional_dataset/stage_2_train_im...,0.183922,0.191654,0.875833,0.825657,0


In [7]:
df[["xmin", "ymin", "xmax", "ymax"]]

,xmin,ymin,xmax,ymax
0,0.148674,0.172123,0.798643,0.755428
1,0.192461,0.047513,0.804094,0.673833
2,0.190476,0.088478,0.845008,0.843607
3,0.143910,0.057366,0.892935,0.900263
4,0.204522,0.045664,0.901839,0.817811
...,...,...,...,...
6025,0.119182,0.214041,0.972144,1.000000
6026,0.096893,0.046766,0.843009,0.906468
6027,0.143966,0.075472,0.820690,0.477594
6028,0.166667,0.002857,0.863263,0.755357


# val only from df

In [8]:
skf = StratifiedKFold(n_splits=5, random_state=1488, shuffle=True)
train_ids = []
val_ids = []
for train_index, valid_index in skf.split(df, df["class"]):
    train_ids.append(train_index)
    val_ids.append(valid_index)

In [9]:
def sklearn_mean_ap(y_pred, y_true):
    return (
        np.mean([average_precision_score(y_true == i, y_pred[:, i]) for i in range(2)])
        * 0.5
    )

In [10]:
class Aug_Crop:
    def augment_image(img):
        h = img.shape[0]
        w = img.shape[1]
        transform = albumentations.Compose(
            [
#                 albumentations.OneOf(
#                     [
#                         albumentations.MotionBlur((3, 5)),
#                         albumentations.MedianBlur(blur_limit=5),
#                         albumentations.GaussianBlur(blur_limit=(3, 5), sigma_limit=0),
#                         albumentations.Blur(blur_limit=(3, 5)),
#                     ],
#                     p=0.1,
#                 ),
#                 albumentations.OneOf(
#                     [
#                         albumentations.GaussNoise(var_limit=[10, 20], mean=1),
#                         albumentations.ImageCompression(
#                             quality_lower=85, quality_upper=100, compression_type=1
#                         ),
#                         albumentations.MultiplicativeNoise(
#                             multiplier=(0.95, 1.05), per_channel=False, elementwise=True
#                         ),
#                         albumentations.Downscale(
#                             scale_min=0.85,
#                             scale_max=0.99,
#                             interpolation=cv2.INTER_LINEAR,
#                         ),
#                     ],
#                     p=0.1,
#                 ),
#                 albumentations.OneOf(
#                     [
#                         albumentations.RandomBrightnessContrast(
#                             brightness_limit=(-0.1, 0.2),
#                             contrast_limit=(-0.1, 0.2),
#                             brightness_by_max=True,
#                         ),
#                         albumentations.augmentations.transforms.Sharpen(
#                             alpha=(0.05, 0.1), lightness=(0.5, 1.0)
#                         ),
#                         albumentations.augmentations.transforms.RandomToneCurve(
#                             scale=0.05
#                         ),
#                     ],
#                     p=0.1,
#                 ),
#                 albumentations.OneOf(
#                     [
#                         albumentations.OpticalDistortion(
#                             distort_limit=0.1,
#                             shift_limit=0.1,
#                             border_mode=1,
#                         ),
#                         albumentations.ElasticTransform(
#                             alpha=2.0,
#                             sigma=2.0,
#                             alpha_affine=2.0,
#                             interpolation=cv2.INTER_LINEAR,
#                             border_mode=1,
#                         ),
#                         albumentations.GridDistortion(
#                             num_steps=5,
#                             distort_limit=0.1,
#                             interpolation=cv2.INTER_LINEAR,
#                             border_mode=1,
#                         ),
#                     ],
#                     p=0.1,
#                 ),
                albumentations.OneOf(
                    [
#                         albumentations.augmentations.crops.transforms.CropAndPad(
#                             px=None,
#                             percent=(-0.01, 0.05),
#                             pad_mode=1,
#                             pad_cval=0,
#                             pad_cval_mask=0,
#                             keep_size=True,
#                             sample_independently=True,
#                             interpolation=cv2.INTER_LINEAR,
#                         ),
#                         albumentations.RandomSizedCrop(
#                             min_max_height=(0.95 * h, 0.95 * w),
#                             height=h,
#                             width=w,
#                             w2h_ratio=1.0,
#                             interpolation=cv2.INTER_LINEAR,
#                         ),
                        albumentations.ShiftScaleRotate(
                            shift_limit=0.0,
                            scale_limit=0.05,
                            rotate_limit=10,
                            interpolation=cv2.INTER_LINEAR,
                            border_mode=1,
                        ),
                        albumentations.HorizontalFlip(),
#                         albumentations.VerticalFlip(),
#                         albumentations.augmentations.Rotate(),
                    ],
                    p=0.1,
                ),
                albumentations.CoarseDropout(
                    max_holes=10,
                    max_height=36,
                    max_width=36,
                    min_holes=1,
                    min_height=6,
                    min_width=6,
                    fill_value=0,
                    p=0.1,
                ),
            ]
        )

        return transform(image=img)["image"]

In [ ]:
for n in range(5):
    val = df.loc[val_ids[n]].sample(frac=1, random_state=base_config[MOD]["SEED"])
    train = pd.concat(
        [
            df.loc[train_ids[n]],
            labels.sample(5000),
        ],
        ignore_index=True,
        axis=0,
    ).sample(frac=1, random_state=base_config[MOD]["SEED"])

    print("\niter#" + str(n) + "\n")

    gen_train = Generator(
        df=train,
        label_columns=["Negative for Pneumonia"],
        batch_size=base_config[MOD]["BATCH_SIZE"],
        seed=base_config[MOD]["SEED"],
        img_size=base_config[MOD]["IMG_SIZE"],
        cache_img_path="/app/_data/train_cropped_npy_380/",
        shuffle=True,
        augment_fn=Aug_Crop,
        crop=True,
    )
    gen_valid = Generator(
        df=val,
        label_columns=["Negative for Pneumonia"],
        batch_size=base_config[MOD]["BATCH_SIZE"],
        seed=base_config[MOD]["SEED"],
        img_size=base_config[MOD]["IMG_SIZE"],
        cache_img_path="/app/_data/train_cropped_npy_380/",
        shuffle=False,
        augment_fn=None,
        crop=True,
    )

    train_sample = gen_train.__getitem__(10)

    plt.figure(figsize=(15, 5))
    for i in range(4):
        plt.subplot(1, 4, i + 1)
        plt.title(train_sample[1][i])
        plt.imshow(train_sample[0][i])
    get_m = GetModel(
        model_name=MOD,
        n_classes=1,
        lr=0.0005,
        activation_func="sigmoid",
        weights="imagenet",
        top_dropout_rate=0.5,
        loss="bce",
        metrics=["acc"],
    )

    model = get_m.get_model()
    policy = keras.mixed_precision.experimental.Policy("mixed_float16")
    keras.mixed_precision.experimental.set_policy(policy)

    callbacks = get_m.make_callback(
        model_path="/app/_data/models/EffB4_2class_cropped_2/",
        model_name="EffB4_2cl_v3_" + str(n) + ".h5",
        tensorboard_path="/app/.tensorboard/EffB4_2class_cropped_222" + str(n),
        patience_ES=10,
        patience_RLR=5,
        factor_LR=0.8,
        metric_for_monitor="val_loss",
        metric_mode="min",
    )

    history = model.fit(
        gen_train,
        validation_data=gen_valid,
        epochs=50,
        steps_per_epoch=train.shape[0] // base_config[MOD]["BATCH_SIZE"],
        validation_steps=val.shape[0] // base_config[MOD]["BATCH_SIZE"],
        verbose=1,
        workers=20,
        max_queue_size=500,
        callbacks=callbacks,
    )

    keras.backend.clear_session()


iter#0



/usr/local/lib/python3.8/dist-packages/matplotlib/text.py:1215: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if s != self._text:


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090, compute capability 8.6
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale
Warning! Model path already exists.
Epoch 1/50
327/327 [==============================] - 375s 1s/step - loss: 0.4968 - acc: 0.7477 - val_loss: 0.5339 - val_acc: 0.7092

Epoch 00001: val_loss improved from inf to 0.53392, saving model to /app/_data/models/EffB4_2class_cropped_2/EffB4_2cl_v3_0.h5
Epoch 2/50
327/327 [==============================] - 319s 967ms/step - loss: 0.3570 - acc: 0.8347 - val_loss: 0.4086 - val_acc: 0.8175

Epoch 00002: val_loss improved from 0.53392 to 0.40860, saving model to /app/_data/models/EffB4_2class_cropped_2/EffB4_2cl_v3_0.h5
Epoch 3/50
327/327 [=====================

KeyboardInterrupt: 